In [7]:
import amazon_dataset
import feature_extraction_text

import numpy as np

In [ ]:
from __future__ import print_function

import datetime
import gc
import math
import os
import subprocess
import sys
import urllib
import zipfile
from functools import lru_cache
from pathlib import Path
from tempfile import gettempdir
from typing import Dict, List, Tuple
from xml.etree.ElementTree import Element as ET_Element
from xml.etree.ElementTree import parse as ET_parse

import albumentations as A
import clip
import cv2
import gdown
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import requests
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras import models
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          DepthwiseConv2D, Dropout, Flatten,
                          GlobalAveragePooling2D, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.figure import Figure
from PIL import Image
from skimage import io
from skimage.transform import resize
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (accuracy_score, f1_score, jaccard_score,
                             precision_score, recall_score)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from torchvision import datasets, transforms
from torchvision.models import ResNet
from torchvision.models.resnet import BasicBlock, Bottleneck
from torchvision.transforms import transforms
from tqdm import tqdm
from tqdm.auto import tqdm

In [8]:
dataset = 'Musical_Instruments'

some_products = amazon_dataset.products_df(dataset)
some_products.sample(n=3)


,asin,description,title,brand,main_cat,rank,price,image_slug,image_url,feature,category,tech_detail
id,,,,,,,,,,,,
101728,B00WEUPR34,"More than just another great JBL sound system,...",JBL 2x EON615 15in Class D Two-Way Multipurpos...,JBL,Musical Instruments,">#83,517 in Musical Instruments (See Top 100 i...",,[51uZqLVgE0L],[https://m.media-amazon.com/images/I/51uZqLVgE...,"[Includes: JBL EON615 Powered 15"" 2-Way System...","[Musical Instruments, Live Sound & Stage, Moni...",None
362,B000068OEW,Hosa's DRA-Series is an excellent cable for S/...,"Hosa DRA-503 RCA to RCA S/PDIF Coax Cable, 3 M...",Hosa,Home Audio & Theater,">#3,926 in Musical Instruments > Recording Equ...",$10.45,[1142tlYEGPL],[https://m.media-amazon.com/images/I/1142tlYEG...,"[Length: 9.8 feet, Connectors: RCA to RCA, Tru...","[Musical Instruments, Live Sound & Stage, Stag...",None
34763,B001NTX914,Banjo/Mandolin Capo with Direct Center-tension...,Paige Acoustic Guitar Body (PBE),PAIGE,Musical Instruments,">#16,650 in Musical Instruments (See Top 100 i...",$17.99,[519xdGpnedL],[https://m.media-amazon.com/images/I/519xdGpne...,"[A sleekly designed, controlled tension capo f...","[Musical Instruments, Instrument Accessories, ...",None


In [9]:
from importlib import reload
reload(feature_extraction_text)

extracted_bert_features = feature_extraction_text.extract_bert_text_features(some_products)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0.00/8.46k [00:00<?, ?product/s]/home/igui/src/VBPR/feature_extraction_text.py:69: GuessedAtParserWarning: No parser was explicitly specifi

In [10]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_bert_features.npz',
    **extracted_bert_features
)

In [11]:
extacted_clip_features = feature_extraction_text.extract_clip_text_features(
    some_products
)

100%|██████████| 8.46k/8.46k [00:47<00:00, 180product/s]


In [12]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_cliptext_features.npz',
    **extacted_clip_features
)